In [ ]:
from datetime import datetime
from pathlib import Path
import pyarrow
import geopandas as gpd 
import pandas as pd 
import matplotlib.pyplot as plt
from shapely.ops import unary_union
import contextily as ctx
import seaborn as sns
from datetime import datetime
import matplotlib.colors as colors
from matplotlib.patches import Patch
from datetime import date

root_dir = Path("~/Desktop/Desktop/epidemiology_PhD/00_repos/").expanduser()

data_dir = root_dir / "los_angeles_2025_fire_disasters_exp/data"

so_cal_counties = ["025", "029", "037", "065", "059", "071", "073", "083", "111", "079"]

crs = 26911

def parse_datetime_string(date_str):
    # split date and time
    date_part = str(date_str)[:7]    # Gets '2025012'
    
    # parse date
    year = date_part[:4]        # '2025'
    day = date_part[5:]         # '012' -> '12'
    month = '01'               # January
    
    # construct datetime string 
    datetime_str = f"{year}-{month}-{day}"
    return pd.to_datetime(datetime_str)

def remove_overlaps(smoke_df):
    smoke_df = smoke_df.to_crs(epsg=3857)
    
    # split by density and make copies of the data
    heavy = smoke_df[smoke_df['Density'] == 'Heavy'].copy()
    medium = smoke_df[smoke_df['Density'] == 'Medium'].copy()
    light = smoke_df[smoke_df['Density'] == 'Light'].copy()
    
    # lets split them up! 
    if not medium.empty and not heavy.empty:
        medium['geometry'] = gpd.GeoSeries(
            # remove heavy areas from medium
            medium.geometry.apply(lambda x: x.difference(heavy.geometry.union_all())),
            index=medium.index,
            crs=medium.crs
        )
    
    if not light.empty:
        if not heavy.empty:
            # remove heavy areas from light
            light['geometry'] = gpd.GeoSeries(
                light.geometry.apply(lambda x: x.difference(heavy.geometry.union_all())),
                index=light.index,
                crs=light.crs
            )
        if not medium.empty:
            # remove medium areas
            light['geometry'] = gpd.GeoSeries(
                light.geometry.apply(lambda x: x.difference(medium.geometry.union_all())),
                index=light.index,
                crs=light.crs
            )
    
    # come back together
    return pd.concat([light, medium, heavy])

In [ ]:
# goal of this script: 
# KP wants to salvage blood samples from its catchment area
# we want to: 
    # 1. identify the zip codes in the kaiser catchment area and determine their lead exposure status using cal enviro screen
    # 2. determine smoke exposure by zip code
    # 3. identify control zip codes that are and are not exposed to smoke and have similar lead exposure percentiles! matching!

In [ ]:
# 1a. identify the zctas in the kp catchment area with geometries!

# read in ca cts and subset to the so cal catchment area 
cts_ca = gpd.read_file(data_dir / "00_raw/tl_2010_06_tract10.shp")
cts_kp = cts_ca[cts_ca['COUNTYFP10'].isin(so_cal_counties)]

# read in zctas 
zctas_us = gpd.read_file(data_dir / "00_raw/tl_2020_us_zcta520.shp")

# intersect zctas with the subsetted cts to get the zcta catchment area
zctas_kp = gpd.sjoin(zctas_us, cts_kp, how="inner", predicate="intersects")
zctas_kp = zctas_kp[["ZCTA5CE20", "geometry"]].rename(columns={"ZCTA5CE20": "zcta"})
zctas_kp = zctas_kp.dissolve(by='zcta', as_index=True)

# where are they? 
zctas_kp.plot()

# this plot shows nothing. need a lot more help to match this up to actual california places

In [ ]:
# where is kaiser, where is so cal, where is california? 
fig, ax = plt.subplots(figsize=(12, 12))
zctas_kp = zctas_kp.to_crs(epsg=3857)
zctas_kp.plot(ax=ax, alpha=0.5, edgecolor='black')

# sos i have no idea where we are in ca

# lets use a basemap with roads and california town and city labels. need all the help we can get.
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Voyager)
ax.set_axis_off()

plt.show()

In [ ]:
# i cant find where things are in CA so lets read in fire boundaries to help match up to sat imagery! 
fire_boundaries = gpd.read_file(data_dir / "00_raw/calfire_boundaries/data_2025_01_17.geojson")
fire_boundaries["poly_DateCurrent"] = fire_boundaries["poly_DateCurrent"].dt.tz_convert('US/Pacific')
fire_boundaries = fire_boundaries[fire_boundaries['poly_DateCurrent'] > '2025-01-06']
fire_boundaries["poly_DateCurrent"] = fire_boundaries["poly_DateCurrent"].dt.date

# lets just pull palisades and eaton since those are the ones in the sat imagery and this is just for my own sanity
fire_boundaries = fire_boundaries[fire_boundaries["incident_name"].isin(["Palisades", "Eaton"])]

# now lets add them to our plot! can we match up california images now? 
fig, ax = plt.subplots(figsize=(12, 12))
zctas_kp = zctas_kp.to_crs(epsg=3857)
fire_boundaries = fire_boundaries.to_crs(epsg=3857)

zctas_kp.plot(ax=ax, alpha=0.5, edgecolor='black')
fire_boundaries.plot(ax=ax, alpha=1, edgecolor='red', facecolor="none")

ctx.add_basemap(ax, source=ctx.providers.CartoDB.Voyager)
ax.set_axis_off()

plt.show()

# still no

In [ ]:
# 1b. pull in the lead exposure data so we can match on that later.
    # going to use percentile and match on percentile bins (by 10 percentage pts)

# read in data 
ces = pd.read_excel(data_dir / "00_raw/calenviroscreen40resultsdatadictionary_F_2021.xlsx")
# NOTE: using cal enviro screen 4.0 for lead data and using lead percentile so it is a relative metric? 
    # https://oehha.ca.gov/calenviroscreen/download-data
# NOTE: these data are at the CT level but are conveniently mapped to zctas 
    # if the same ZCTA is present in multiple rows, we'll take the mean of the values across rows for now


ces_cols = ["ZIP", "Lead"]
# NOTE: Joan, do you have a pref between using lead percentile and lead? see data dict for definitions
ces = ces[ces_cols].rename(columns={"ZIP": "zcta", "Lead": "lead"})

# some zip-cts have nas so drop those: 
ces = ces.dropna(subset=["zcta", "lead"])

# aggregate to zcta level
ces = ces.groupby("zcta").mean().reset_index()
ces["zcta"] = ces["zcta"].astype(str).str.zfill(5)

# subset ces data to kp zctas and merge them on: 
kp_ces = zctas_kp.merge(ces, on="zcta")

# make percentile bins for matching: 
kp_ces["lead_bin"] = pd.qcut(kp_ces["lead"], q=10, labels=False)

# plot by lead bin
fig, ax = plt.subplots(figsize=(10, 7))
kp_ces = kp_ces.to_crs(epsg=3857)

# Store the plot output
plot = kp_ces.plot(ax=ax, alpha=0.5, edgecolor=None, column="lead", legend=True)

ctx.add_basemap(ax, source=ctx.providers.CartoDB.Voyager)
ax.set_axis_off()

plt.show()

In [ ]:
# 2a. clean up smoke data

# read in smoke files for jan 7-12, 2025
smoke_dir = data_dir / "00_raw/smoke_plume"
smoke_files = list(smoke_dir.glob("*/*.shp"))

# read in all files and create a date var for each: 
smoke = pd.concat([gpd.read_file(f) for f in smoke_files])
smoke["date"] = smoke["Start"].apply(parse_datetime_string)

# subset to only the density, geometry, and date
smoke = smoke[["date", "geometry", "Density"]]

# subset smoke to kp zctas
smoke_kp = smoke.to_crs(epsg=3857)
smoke_kp = gpd.sjoin(smoke_kp, zctas_kp, how="inner", predicate="intersects")

# dissolve by date and density
smoke = smoke.dissolve(by=['date', 'Density'], as_index=True).reset_index()

# convert back to gdf
smoke = gpd.GeoDataFrame(smoke, geometry='geometry')

In [ ]:
# 2b. map it! 

# colors
okeefe = ["#fbe3c2", "#f2c88f", "#ecb27d", "#e69c6b", "#d37750", "#b9563f", "#92351e"]
density_colors = {
    'Light': okeefe[0],    # lightest
    'Medium': okeefe[2],   # medium
    'Heavy': okeefe[4]     # darkest
}

fig, axs = plt.subplots(1, 6, figsize=(24, 4))
axs = axs.flatten()

# bounds of the entire dataset
total_bounds = zctas_kp.total_bounds

for i, date in enumerate(sorted(smoke["date"].unique())):
    ax = axs[i]
    
    # subset to 
    smoke_date = smoke[smoke["date"] == date]
    
    # remove overlaps so that we can get heavy on top! 
    smoke_date_no_overlap = remove_overlaps(smoke_date)
    
    # base map layers
    kp_ces.plot(ax=ax, alpha=0.5, edgecolor="black")
    
    # plot each density level separately
    for density in ['Light', 'Medium', 'Heavy']:
        density_data = smoke_date_no_overlap[smoke_date_no_overlap['Density'] == density]
        if not density_data.empty:
            density_data.plot(
                ax=ax,
                color=density_colors[density],
                alpha=0.75,
                edgecolor=None
            )
    
    # manual legend
    legend_elements = [
        Patch(facecolor=density_colors['Heavy'], label='Heavy', alpha=0.75),
        Patch(facecolor=density_colors['Medium'], label='Medium', alpha=0.75),
        Patch(facecolor=density_colors['Light'], label='Light', alpha=0.75)
    ]
    ax.legend(handles=legend_elements, loc='upper right')
    
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Voyager)
    ax.set_axis_off()
    ax.set_title(pd.to_datetime(date).strftime('%Y-%m-%d'))
    
    # set the same bounds for each subplot
    ax.set_xlim(total_bounds[0], total_bounds[2])
    ax.set_ylim(total_bounds[1], total_bounds[3])

plt.tight_layout()
plt.show()

In [ ]:
# colors
okeefe = ["#fbe3c2", "#f2c88f", "#ecb27d", "#e69c6b", "#d37750", "#b9563f", "#92351e"]
density_colors = {
    'Light': okeefe[0],    # lightest
    'Medium': okeefe[2],   # medium
    'Heavy': okeefe[4]     # darkest
}

# create figure with extra space at bottom for the lead legend
fig, axs = plt.subplots(1, 6, figsize=(24, 5), constrained_layout=True)
axs = axs.flatten()

# bounds of the entire dataset
total_bounds = zctas_kp.total_bounds

# keep track of the lead plot for the legend
lead_plot = None

for i, date in enumerate(sorted(smoke["date"].unique())):
    ax = axs[i]
    
    smoke_date = smoke[smoke["date"] == date]
    smoke_date_no_overlap = remove_overlaps(smoke_date)
    
    # base map layers - only show legend for lead in the last plot
    lead_plot = kp_ces.plot(
        ax=ax, 
        alpha=0.5, 
        edgecolor=None, 
        column="lead", 
        legend=False  # Turn off individual legends
    )
    
    # plot each density level separately
    for density in ['Light', 'Medium', 'Heavy']:
        density_data = smoke_date_no_overlap[smoke_date_no_overlap['Density'] == density]
        if not density_data.empty:
            density_data.plot(
                ax=ax,
                color=density_colors[density],
                alpha=0.75,
                edgecolor=None
            )
    
    # manual legend for smoke
    legend_elements = [
        Patch(facecolor=density_colors['Heavy'], label='Heavy', alpha=0.75),
        Patch(facecolor=density_colors['Medium'], label='Medium', alpha=0.75),
        Patch(facecolor=density_colors['Light'], label='Light', alpha=0.75)
    ]
    ax.legend(handles=legend_elements, loc='upper right')
    
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Voyager)
    ax.set_axis_off()
    ax.set_title(pd.to_datetime(date).strftime('%Y-%m-%d'))
    
    ax.set_xlim(total_bounds[0], total_bounds[2])
    ax.set_ylim(total_bounds[1], total_bounds[3])

# manual legend for lead
cax = fig.add_axes([0.3, 0.05, 0.4, 0.04])  # [left, bottom, width, height]
sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=plt.Normalize(vmin=kp_ces['lead'].min(), vmax=kp_ces['lead'].max()))
cbar = fig.colorbar(sm, cax=cax, orientation='horizontal')
cbar.set_label('Lead Level', fontsize=10)
cbar.set_label('Lead Level', fontsize=16)
cbar.ax.tick_params(labelsize=14)

plt.show()

In [ ]:
# 2c. 
#---------------------
# first find all zctas exposed to heavy smoke at least once 

# find all zctas exposed to heavy smoke at least twice 
heavy_zcta_counts = smoke_kp[smoke_kp['Density'] == 'Heavy']['zcta'].value_counts()
frequent_heavy_zctas = heavy_zcta_counts[heavy_zcta_counts >= 3].index.tolist()

print("ZCTAs with heavy smoke at least twice:", frequent_heavy_zctas)
print("Count:", len(frequent_heavy_zctas))

In [ ]:
# 2d. 
#---------------------
# second find all zctas exposed to no smoke ever 

# find all zctas exposed to no smoke ever by looking at which zctas in zctas_kp do not show up in smoke_kp
zctas_with_smoke = smoke_kp['zcta'].unique()
all_zctas = zctas_kp.reset_index(names = 'zcta')
never_in_smoke = all_zctas[~all_zctas['zcta'].isin(zctas_with_smoke)]
never_in_smoke

print("\nZCTAs never intersecting with smoke:")
print(never_in_smoke['zcta'].tolist())
print("Count:", len(never_in_smoke))

#---------------------
# merge on kp pop to make sure there are enough people 
pop = pd.read_csv(data_dir / "00_raw/kpsc_zcta_counts.csv")
pop['zcta'] = pop['zcta'].astype(str).str.zfill(5)
pop = pop.drop(columns = 'classification')

# make a dataframe of all the zctas with heavy smoke and then all the zctas with no smoke and # of people in each
heavy = pop[pop['zcta'].isin(frequent_heavy_zctas)]

# make col that is 1 if heavy smoke and 0 if no smoke
heavy['smoke'] = 1

# make a dataframe of all zctas with no smoke and # of people in each
no_smoke = pop[pop['zcta'].isin(never_in_smoke['zcta'])]
no_smoke['smoke'] = 0

# concat the 2 dataframes
all = pd.concat([heavy, no_smoke])

In [ ]:
# 3. 
# merge lead bins
smoke_lead_zctas = all.merge(kp_ces, on="zcta")

# match cases (smoke=1) and controls (smoke=0) on lead bin
smoke_lead_zctas = smoke_lead_zctas.sort_values(by="lead")

# get counts of cases/controls per lead_bin
summary = (smoke_lead_zctas
    .groupby(['lead_bin', 'smoke'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

summary.columns = ['lead_bin', 'no_smoke', 'heavy_smoke']

In [ ]:
# final choices! 
#---------------------
# lets pick the lead bins with the most overlap between no smoke and heavy smoke based on lead bin! those will be priority 1 
priority_lead_bins = summary[summary['no_smoke'] > 0]
priority_lead_bins = priority_lead_bins['lead_bin'].tolist()

# if a zcta is a priority zcta, make it priority 1, otherwise priority 2. make priority col 
blood_sample_zctas = smoke_lead_zctas.copy()
blood_sample_zctas['priority'] = 2
blood_sample_zctas.loc[blood_sample_zctas['lead_bin'].isin(priority_lead_bins), 'priority'] = 1

# drop geometry col so we can write out csvs 
blood_sample_zctas = blood_sample_zctas.drop(columns = 'geometry')

# rename smoke to heavy smoke so it is more clear 
blood_sample_zctas = blood_sample_zctas.rename(columns = {'smoke': 'heavy_smoke'})
blood_sample_zctas

In [ ]:
# write out a csv of the zctas to sample blood from and the summary for joan
# write out a csv of just zctas and priorities for kp 
blood_sample_zctas.to_csv(data_dir / "01_intermediate/blood_sample_zctas.csv", index=False)
blood_sample_zctas[['zcta', 'priority']].to_csv(data_dir / "01_intermediate/kp_blood_sample_zctas.csv", index=False)
summary.to_csv(data_dir / "01_intermediate/blood_sample_zctas_summary.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np

def create_lead_bin_summary(df):
    """
    Creates a summary table of lead bins and their corresponding statistics.
    
    Parameters:
    df: DataFrame containing 'lead' and 'lead_bin' columns
    
    Returns:
    DataFrame with bin summary statistics
    """
    # summary statistics for each bin
    bin_summary = df.groupby('lead_bin').agg({
        'lead': [
            ('min', 'min'),
            ('max', 'max'),
            ('mean', 'mean'),
            ('median', 'median'),
            ('count', 'count')
        ]
    }).round(2)
    
    # flatten column names
    bin_summary.columns = bin_summary.columns.get_level_values(1)
    
    # make lead_bin a column
    bin_summary = bin_summary.reset_index()
    
    # add bin range column for easier interpretation
    bin_summary['bin_range'] = bin_summary.apply(
        lambda x: f"{x['min']:.2f} - {x['max']:.2f}",
        axis=1
    )
    
    # reorder columns
    final_columns = ['lead_bin', 'bin_range', 'min', 'max', 'mean', 'median', 'count']
    bin_summary = bin_summary[final_columns]
    
    return bin_summary

# make summary
lead_summary = create_lead_bin_summary(kp_ces)

# save summary
lead_summary.to_csv(data_dir / "01_intermediate/lead_summary.csv", index=False)